In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
import datetime as DT 
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, f1_score
import seaborn as sb

In [2]:
data = pd.read_csv(r"E:\ETLHIVE Training\ML Problem\TVS\Data.csv",low_memory=False)

In [3]:
#sb.pairplot(data)

In [4]:
#Converting DOB to age

now = pd.Timestamp('now')
data["V3"] = pd.to_datetime(data["V3"])
data['V3'] = data['V3'].where(data['V3'] < now, data['V3'] -  np.timedelta64(100, 'Y'))   # 2
data['V3'] = (now - data['V3']).astype('<m8[Y]')    # 3

In [5]:
def repWhiteSpacesByNull(strValue):
    strValue = re.sub(r"\W+","",strValue)
    return strValue

In [6]:
data["V12"] = data["V12"].astype("str")
#data["V3"] = data["V3"].astype("str")
data["V12"] = data["V12"].apply(repWhiteSpacesByNull)
#tempData["V3"] = tempData["V3"].apply(repWhiteSpacesByNull)

In [8]:
X = data[data.columns[~data.columns.isin(["V12","V24","V14","V1","V3","V32"])]]
Y = data[["V32"]]

In [9]:
X = X.apply(LabelEncoder().fit_transform)
X = pd.DataFrame(RobustScaler().fit_transform(X))

In [10]:
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size = 0.20,random_state = 25)

In [31]:
xg_reg = xgb.XGBClassifier(max_depth = 10,objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.05, n_estimators = 100,min_child_weight = 2, gamma = 0.3, subsample = 0.6,scale_pos_weight = 9)
model = xg_reg.fit(xtrain,ytrain)
pred = model.predict(xtest)


E:\Anconda\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anconda\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\Anconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
roc_auc = roc_auc_score(ytest,pred)
acc = accuracy_score(ytest,pred)
f1_Score = f1_score(ytest,pred,average='weighted')

In [36]:
print("ROC_AUC_SCORE: ",roc_auc)
print("Accuracy Score: ",acc)
print("F1_Score: ",f1_Score)

ROC_AUC_SCORE:  0.6417406209525204
Accuracy Score:  0.8484227268710789
F1_Score:  0.868993806496849


In [37]:
creport = classification_report(ytest,pred)
print(creport)

             precision    recall  f1-score   support

          0       0.95      0.88      0.92     52477
          1       0.21      0.40      0.28      4108

avg / total       0.90      0.85      0.87     56585

